In [2]:
import sys
import os
import re
import json

sys.path.append('..')

import lcatools


##
def show(res):
    for i, k in enumerate(res):
        print('[%2d] %s ' % (i, k))

from lcatools.interact import *



In [3]:
from lcatools.foreground.manager import ForegroundManager
F = ForegroundManager('fragments', cfs=None)
#from lcatools.foreground.interface import ForegroundInterface
#F = ForegroundInterface('fragments')


ForegroundArchive archive added in position 0
IlcdLcia archive added in position 1
JSON archive added in position 2
JSON archive added in position 3
JSON archive added in position 4
JSON archive added in position 5
JSON archive added in position 6
JSON archive added in position 7
JSON archive added in position 8
JSON archive added in position 9
JSON archive added in position 10
Generating flow-quantity database...
finished... (0.78 s)


In [4]:
F.show(loaded=False)


LCA Catalog with the following archives:
X [ 0] FG     : ForegroundArchive with 24 entities at fragments
  [ 1] ELCD   : /data/LCI/ELCD/ELCD3.2.zip
  [ 2] LCIA   : /data/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz
  [ 3] EI-LCIA: /data/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz
  [ 4] EI-U   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_undefined_xlsx.json.gz
  [ 5] EI_A   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_apos_xlsx.json.gz
  [ 6] EI_Q   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_consequential_xlsx.json.gz
  [ 7] EI_C   : /data/GitHub/lca-tools-datafiles/catalogs/ecoinvent_3.2_cut-off_xlsx.json.gz
  [ 8] USLCI  : /data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz
  [ 9] GaBi-P : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_professional-database-2016.json.gz
  [10] GaBi-X : /data/GitHub/lca-tools-datafiles/catalogs/gabi_2016_all-extensions.json.gz


## select quantities

In [5]:
F.selected = set(F.search(2, 'quantity', Name='usetox', show=True))

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz:
Found Extension: zip
54 new quantity entities added (54 total)
1256 new flow entities added (1256 total)
0 new process entities added (0 total)
[ 0] (LCIA) {q} ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] 
[ 1] (LCIA) {q} ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA] 
[ 2] (LCIA) {q} ILCD2011; Non-cancer human health effects; midpoint; CTUh; USEtox [LCIA] 
[ 3] (LCIA) {q} ILCD2011; Cancer human health effects; endpoint; DALY; USEtox [LCIA] 
[ 4] (LCIA) {q} ILCD2011; Cancer human health effects; midpoint; CTUh; USEtox [LCIA] 


In [7]:
for f in F.selected: 
    F.add_to_foreground(f)

Add to foreground: (LCIA) {q} ILCD2011; Non-cancer human health effects; midpoint; CTUh; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Cancer human health effects; endpoint; DALY; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Cancer human health effects; midpoint; CTUh; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]


## build a fragment

In [9]:
F.load('USLCI')

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz:
Found Extension: zip
19 new quantity entities added (19 total)
4176 new flow entities added (4176 total)
701 new process entities added (701 total)


In [10]:
selected = F.search('USLCI', 'flow', Name='poly.*terephth')

show(selected)


[ 0] (USLCI) {f} Polyethylene terephthalate, resin, at plant, CTR [Chemical Manufacturing, All Other Basic Organic Chemical Manufacturing] 
[ 1] (USLCI) {f} Recovered energy, for Polyethylene terephthalate, resin, at plant, CTR [Chemical Manufacturing, All Other Basic Organic Chemical Manufacturing] 
[ 2] (USLCI) {f} Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing] 
[ 3] (USLCI) {f} CUTOFF Polyethylene terephthalate, granulate, amorphous, at plant [CUTOFF Flows] 


In [11]:
F.add_to_foreground(selected[0])
F.add_to_foreground(selected[2])

Add to foreground: (USLCI) {f} Polyethylene terephthalate, resin, at plant, CTR [Chemical Manufacturing, All Other Basic Organic Chemical Manufacturing]
Add to foreground: (USLCI) {f} Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing]


In [12]:
F[0].flows()[1]['Name']

'Polyethylene terephthalate, resin, at plant'

In [13]:
F[0].create_fragment(F[0].flows()[1], 'Input', 'PET from USLCI')

LcFragment - Name: PET from USLCI:


In [14]:
[str(f) for f in F[0].fragments(show_all=True)]

['( ** ref) <--  be8c7cc <--  --: : Polyethylene terephthalate, resin, at plant']

In [15]:
frag = next(f for f in F[0].fragments())

In [16]:
F.auto_terminate('USLCI', frag)

In [17]:
[str(f) for f in F[0].fragments(show_all=True)]

['( ** ref) <--  be8c7cc <--  #   : Polyethylene terephthalate, resin, at plant']

In [18]:
F.save()

Saving foreground


In [19]:
F.lcia(frag.termination(None).term_node)

    P0    LCIA Method
-------------------------------
         0  ILCD2011; Non-cancer human health effects; midpoint; CTUh; USEtox [LCIA]
         0  ILCD2011; Cancer human health effects; endpoint; DALY; USEtox [LCIA]
         0  ILCD2011; Cancer human health effects; midpoint; CTUh; USEtox [LCIA]
         0  ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA]
         0  ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]

P0: (USLCI) {p} Polyethylene terephthalate, resin, at plant [RNA]


In [20]:
F.load_lcia_cfs(2)

0 unmatched flows found from source LCIA... 



In [21]:
F.lcia(frag.termination(None).term_node)

    P0    LCIA Method
-------------------------------
  7.04e-09  ILCD2011; Non-cancer human health effects; midpoint; CTUh; USEtox [LCIA]
  2.24e-12  ILCD2011; Cancer human health effects; endpoint; DALY; USEtox [LCIA]
  1.95e-13  ILCD2011; Cancer human health effects; midpoint; CTUh; USEtox [LCIA]
   1.9e-08  ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA]
     0.213  ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]

P0: (USLCI) {p} Polyethylene terephthalate, resin, at plant [RNA]


In [22]:
F.build_child_flows(frag)

child fragment - no termination found: CUTOFF Disposal, solid waste, unspecified, to waste-to-energy
child fragment - no termination found: CUTOFF Disposal, solid waste, unspecified, to municipal incineration
child fragment - no termination found: CUTOFF Disposal, solid waste, unspecified, to sanitary landfill
child fragment - no termination found: CUTOFF Ethylene glycol, at plant
child fragment - no termination found: Recovered energy


In [23]:
F[0].flows()

In [24]:
term = frag.termination(None)

In [25]:
int_exch = [x for x in F._filter_exch(term.term_node, elem=False)
            if not (x.flow == term.term_flow and x.direction == term.direction)]


In [26]:
[str(x) for x in int_exch]

[' Input:     [0.0128 [l]] Diesel, combusted in industrial equipment [Utilities, Steam and Air-Conditioning Supply]',
 ' Input:     [0.000655 [t*km]] Transport, barge, diesel powered [Water Transportation, Inland Water Freight Transportation]',
 ' Input:     [0.0372 [kg]] Acetic acid, at plant [Chemical Manufacturing, All Other Basic Organic Chemical Manufacturing]',
 ' Input:     [8.76e-05 [t*km]] Transport, pipeline, natural gas [Utilities, Fossil Fuel Electric Power Generation]',
 ' Input:     [0.0981 [m3]] Natural gas, combusted in industrial boiler [Utilities, Steam and Air-Conditioning Supply]',
 ' Input:     [0.521 [kg]] Paraxylene, at plant [Chemical Manufacturing, Petrochemical Manufacturing]',
 ' Input:     [0.00059 [kg]] CUTOFF Disposal, solid waste, unspecified, to waste-to-energy [CUTOFF Flows]',
 ' Input:     [1.61 [t*km]] Transport, train, diesel powered [Rail Transportation, Rail Transportation]',
 ' Input:     [0.00031 [kg]] CUTOFF Disposal, solid waste, unspecified, t

In [27]:
[str(f) for f in F[0].fragments(background=False, show_all=True)]

['(be8c7cc) <--  4da1902 <--  #   : Bituminous coal, combusted in industrial boiler',
 '(be8c7cc) <--  0927188 <--  --: : CUTOFF Disposal, solid waste, unspecified, to waste-to-energy',
 '(be8c7cc) <--  4e717b4 <--  #   : Transport, barge, diesel powered',
 '(be8c7cc) <--  0fa4a16 <--  #   : Paraxylene, at plant',
 '(be8c7cc) <--  033e693 <--  #   : Acetic acid, at plant',
 '(be8c7cc)  ==> 2f26767  ==> --: : Recovered energy',
 '(be8c7cc) <--  026552f <--  #   : Transport, train, diesel powered',
 '(be8c7cc) <--  a0c84a5 <--  #   : Electricity, at cogen, for natural gas turbine',
 '(be8c7cc) <--  fed71a5 <--  #   : Electricity, at grid, US, 2008',
 '( ** ref) <--  be8c7cc <--  #   : Polyethylene terephthalate, resin, at plant',
 '(be8c7cc) <--  c3ae97d <--  --: : CUTOFF Disposal, solid waste, unspecified, to municipal incineration',
 '(be8c7cc) <--  14b3a7b <--  --: : CUTOFF Disposal, solid waste, unspecified, to sanitary landfill',
 '(be8c7cc) <--  d392c5c <--  #   : Transport, barge,

In [28]:
F.save()

Saving foreground


In [29]:
F.search(0,'fragment', show=True)

[ 0] (FG) {f} (be8c7cc) <--  4da1902 <--  #   : Bituminous coal, combusted in industrial boiler 
[ 1] (FG) {f} (be8c7cc) <--  0927188 <--  --: : CUTOFF Disposal, solid waste, unspecified, to waste-to-energy 
[ 2] (FG) {f} (be8c7cc) <--  4e717b4 <--  #   : Transport, barge, diesel powered 
[ 3] (FG) {f} (be8c7cc) <--  0fa4a16 <--  #   : Paraxylene, at plant 
[ 4] (FG) {f} (be8c7cc) <--  033e693 <--  #   : Acetic acid, at plant 
[ 5] (FG) {f} (be8c7cc)  ==> 2f26767  ==> --: : Recovered energy 
[ 6] (FG) {f} (be8c7cc) <--  026552f <--  #   : Transport, train, diesel powered 
[ 7] (FG) {f} (be8c7cc) <--  a0c84a5 <--  #   : Electricity, at cogen, for natural gas turbine 
[ 8] (FG) {f} (be8c7cc) <--  fed71a5 <--  #   : Electricity, at grid, US, 2008 
[ 9] (FG) {f} ( ** ref) <--  be8c7cc <--  #   : Polyethylene terephthalate, resin, at plant 
[10] (FG) {f} (be8c7cc) <--  c3ae97d <--  --: : CUTOFF Disposal, solid waste, unspecified, to municipal incineration 
[11] (FG) {f} (be8c7cc) <--  14b3a

In [13]:
F.search(8, 'flow', Name='nitrogen oxides', show=True)

[ 0] (USLCI) {f} Nitrogen oxides [air, low population density] 
[ 1] (USLCI) {f} Nitrogen oxides [air, low. pop.] 
[ 2] (USLCI) {f} Nitrogen oxides  [air, unspecified] 
[ 3] (USLCI) {f} Nitrogen oxides [air, high population density] 
[ 4] (USLCI) {f} Nitrogen oxides [water, unspecified] 
[ 5] (USLCI) {f} Nitrogen oxides [air, stratosphere] 
[ 6] (USLCI) {f} Nitrogen oxides [air, low population density, long-term] 


In [16]:
F.elementary(frag.termination(None).term_node)

Elementary exchanges:
Output:     [4.1e-05 [kg]] Xylene [air, unspecified]
Output:     [9.7e-10 [kg]] Antimony [water, unspecified]
Output:     [5.5e-06 [kg]] Zinc [water, unspecified]
Output:     [4.4e-05 [kg]] TOC, Total Organic Carbon [water, unspecified]
Output:     [0.00568 [kg]] Carbon monoxide [air, unspecified]
Output:     [9.7e-10 [kg]] Aluminum [water, unspecified]
Output:     [0.00094 [kg]] Organic substances, unspecified [air, unspecified]
Output:     [5.2e-05 [kg]] Nitrogen oxides  [air, unspecified]
Output:     [0.00015 [kg]] Particulates, unspecified [air, unspecified]
Output:     [9.4e-05 [kg]] Aldehydes [Elementary Flows]
Output:     [0.0003 [kg]] BOD5, Biological Oxygen Demand [water, unspecified]
Output:     [4.5e-09 [kg]] Metallic ions, unspecified [water, unspecified]
Output:     [5.4e-05 [kg]] Suspended solids, unspecified [water, unspecified]
Output:     [0.00016 [kg]] Methane  [air, unspecified]
Output:     [9.7e-10 [kg]] Iron [water, unspecified]
Output:     [0

In [11]:
frag.traverse(lambda x: F.child_flows(x), 1, None)

KeyError: None

In [11]:
q = F[0].lcia_methods(Name='Non-Cancer.*mid')[0]

In [12]:
q

In [7]:
term = F._catalog.terminate_fragment('USLCI', frag)[0]

In [13]:
F.show_detailed_lcia(frag.termination(None).term_node, q, show_all=True)

ILCD2011; Non-cancer human health effects; midpoint; CTUh; USEtox [LCIA] Item(s)
------------------------------------------------------------
   0.00011 x       ---- =          0 Ammonia [water, unspecified]
   9.7e-10 x       ---- =          0 Aluminum [water, unspecified]
   0.00076 x       ---- =          0 COD, Chemical Oxygen Demand [water, unspecified]
     3e-05 x       ---- =          0 Dissolved solids [water, unspecified]
   5.1e-05 x       ---- =          0 Acetic acid [air, unspecified]
   0.00568 x       ---- =          0 Carbon monoxide [air, unspecified]
   0.00028 x       ---- =          0 NMVOC, non-methane volatile organic compounds, unspecified  [Elementary Flows]
     4e-05 x       ---- =          0 Methyl acetate [air, unspecified]
   3.6e-09 x       ---- =          0 Phenol compounds, unspecified [water, unspecified]
   5.2e-05 x       ---- =          0 Nitrogen oxides  [air, unspecified]
   9.7e-10 x       ---- =          0 Iron [water, unspecified]
   0.00016 x 

In [13]:
F.find_flowable('^Methyl acetate$')

{133}

In [14]:
[str(f.characterization) for f in F._flowdb.all_cfs(F._flowdb.flowables.search('^zinc$').pop())]

['2.11e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '3.86e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '2.11e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '1.68e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '3.27e-15 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '1.67e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '2.11e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '3.86e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '1.68e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '1.69e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '1.69e+04 [GLO] [m3*a] ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]',
 '3.86e+04

In [8]:
F.cfs_for_flowable_grid('^water$')

Water [007732-18-5]
Characterization Factors
 
Locations  |   C0   |   C1   |   C2    Compartment
--------------------------------------------------
      GLO         1   --       --      []
      GLO         1   --       --      ['Resources', 'Resources from water', 'Renewable material resources from water']
      GLO         1   --       --      ['Resources', 'Resources from water', 'Renewable material resources from water']
      GLO    --            1   --      ['Resources', 'Resources from water', 'Renewable material resources from water']


TypeError: float() argument must be a string or a number, not 'dict'

In [8]:
term

ExchangeRef(catalog=<lcatools.catalog.CatalogInterface object at 0x7f40950604e0>, index='USLCI', exchange=<lcatools.exchanges.ExchangeValue object at 0x7f40928dad68>)

In [9]:
frag.term_from_exch(term)

In [12]:
frag.serialize()

{'Comment': '',
 'Direction': 'Input',
 'Flow': 'Polyethylene terephthalate, resin, at plant',
 'Name': 'USLCI - PET',
 'StageName': '',
 'direction': 'Input',
 'entityId': '572813aa-b8a3-4034-bff1-ff7e469d8726',
 'entityType': 'fragment',
 'exchangeValues': {0: None, 1: 1.0},
 'externalId': '572813aa-b8a3-4034-bff1-ff7e469d8726',
 'flow': '2a6b644c-519c-32e9-a7d8-521c3011df09',
 'isBackground': False,
 'isBalanceFlow': False,
 'isPrivate': False,
 'origin': 'fragments',
 'parent': 'None',
 'tags': {'Comment': '',
  'Direction': 'Input',
  'Flow': 'Polyethylene terephthalate, resin, at plant',
  'Name': 'USLCI - PET',
  'StageName': ''},
 'terminations': {None: {'entityId': '42550686-82c9-3e19-b47d-fadc0037f6a8',
   'source': '/data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz'}}}

In [5]:
F.load(2)

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz:
Found Extension: zip
54 new quantity entities added (54 total)
1256 new flow entities added (1256 total)
0 new process entities added (0 total)


In [17]:
F._flowdb.import_cfs(2)

[]

In [18]:
F.unmatched_flows[2] = F._flowdb.import_cfs(2)

In [7]:
for f in F.selected:
    F.add_to_foreground(f)


Add to foreground: (LCIA) {q} ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Cancer human health effects; endpoint; DALY; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Non-cancer human health effects; midpoint; CTUh; USEtox [LCIA]
Add to foreground: (LCIA) {q} ILCD2011; Cancer human health effects; midpoint; CTUh; USEtox [LCIA]


In [91]:
F.add_selection()

IndexError: pop from empty list

In [23]:
[str(f) for f in F[2].flows() if q in f._characterizations.keys()]

NameError: name 'q' is not defined

In [25]:
F._flowdb.factors_for_quantity(F[0].lcia_methods()[3])

ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA]
Characterization Factors
 
CAS Number   |   C0   |   C1   |   C2   |   C3   |   C4   |   C5   |   C6   |   C7   |   C8   |   C9   |   C10  |   C11  |   C12   Flowable
-------------------------------------------------------------------------------------------------------------------------------------------
       None    --       --       --       --       --       --      2.3e-06   --       --       --       --       --       --      (2,4-dichlorophenoxy)acetic acid
       None    --       --       --       --     6.25e-06   --       --       --       --       --       --       --       --      (2,4-dichlorophenoxy)acetic acid
000079-00-5    --       --       --       --       --       --       --       --       --     7.97e-06   --       --       --      1,1,2-trichloroethane
000106-93-4    --     3.43e-07   --       --       --       --       --       --       --       --     2.85e-07   --       --      1,2-dibr

In [36]:
q = F[0].lcia_methods()[3]

In [30]:
F._flowdb._q_dict[q.get_uuid()]

{4,
 15,
 23,
 24,
 26,
 27,
 32,
 33,
 37,
 41,
 47,
 51,
 52,
 53,
 54,
 58,
 59,
 60,
 61,
 62,
 66,
 67,
 69,
 71,
 72,
 75,
 77,
 78,
 79,
 82,
 92,
 93,
 94,
 102,
 110,
 118,
 119,
 120,
 121,
 123,
 128,
 129,
 130,
 132,
 134,
 135,
 140,
 141,
 148,
 151,
 152,
 155,
 158,
 163,
 164,
 166,
 174,
 192,
 196,
 197,
 198,
 199,
 201,
 202,
 204,
 248,
 249,
 254,
 255,
 257,
 258,
 259,
 261,
 262,
 264,
 266,
 271,
 277,
 278,
 280,
 283,
 286,
 288,
 293,
 294,
 295,
 300,
 304,
 309,
 313,
 314,
 315,
 317,
 319,
 320,
 323,
 341,
 342,
 343,
 344,
 345,
 529,
 531,
 532,
 541,
 546,
 548,
 552,
 555,
 563,
 564,
 567,
 569,
 570,
 571,
 572,
 573,
 574,
 575,
 577,
 579,
 581,
 590,
 599,
 600,
 603,
 604,
 605,
 608,
 609,
 611,
 613,
 615,
 616,
 620,
 624,
 625,
 631,
 632,
 634,
 637,
 651,
 652,
 654,
 655,
 656,
 661,
 662,
 664,
 665,
 666,
 686,
 703,
 742,
 745,
 755,
 997,
 1097,
 1197,
 1506,
 1536,
 1603,
 1607,
 1612,
 1613,
 1614,
 1615,
 1616,
 1617,
 1618,
 

In [31]:
F._flowdb._f_dict[(fb, q)]

NameError: name 'fb' is not defined

In [32]:
q = F[0].lcia_methods()[0].get_uuid()
cls_0 = [F._flowdb._f_dict[(fb, q)] for fb in F._flowdb._q_dict[q]]

In [33]:
[cf.characterization.locations() for i in cls_0 for k, v in i._dict.items() for cf in v]

[dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['GLO']),
 dict_keys(['

In [37]:
res = F.fg_lcia(frag.termination(None)._process_ref, q)[q.get_uuid()]

In [38]:
[str(k.factor) for k in res.LciaScores['42550686-82c9-3e19-b47d-fadc0037f6a8'].LciaDetails]

['Particulates, unspecified [air, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 'COD, Chemical Oxygen Demand [water, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 'Aluminum [water, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 'Ammonia [water, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 'Carbon dioxide, fossil  [air, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 'Methane  [air, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 '[2.98e-07 [a]] ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA]',
 'Suspended solids, unspecified [water, unspecified] has ILCD2011; Non-cancer human health effects; endpoint; DALY; USEtox [LCIA] [a]',
 'Phosphates [water, unspecified] has 

In [39]:
res.total()

1.904317151e-08

In [128]:
[k.factor.locations() for k in next(i for i in res.LciaScores.values()).LciaDetails]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [77]:
r1 = [k for k in res.LciaScores['42550686-82c9-3e19-b47d-fadc0037f6a8'].LciaDetails if k.exchange.flow['Name'] == 'Nitrogen oxides ']

In [78]:
str(r1[0].exchange)

'Output:     [5.2e-05 [kg]] Nitrogen oxides  [air, unspecified]'

In [86]:
r1[0].exchange.flow.profile()

Nitrogen oxides  [air, unspecified]
[1 [kg]] EcoSpold Quantity kg
Nitrogen oxides  [air, unspecified] has ILCD2011; Acidification terrestrial; endpoint; PNOF; Van Zelm et al 2007 [LCIA] [Items*a]


In [80]:
F._flowdb._parse_flow(r1[0].exchange.flow)[0]

{16}

In [83]:
F._flowdb.flowables[16]

{'011104-93-1',
 '191b44d4-90c9-465a-8802-93a651b4fd52',
 '2f89fbbd-e428-4de1-8c33-9dd66e53310c',
 'Nitrogen oxides',
 'b48a94a6-1707-4e6b-8efd-3e4084921ca5',
 'e575ebc3-0a3b-4c38-9a2a-13e42c72553b',
 'f79d0f8f-2b0e-49cb-bed0-b1ea0fbd8625',
 'nitrogen oxides'}

In [81]:
16 in F._flowdb._q_dict[q]

False

In [ ]:
F._flowdb._f_dict[(1)]

In [57]:
F.show_detailed_lcia(frag.termination(None)._process_ref, next(l for l in F[0].lcia_methods()), show_all=True)

ILCD2011; Ecotoxicity freshwater; midpoint; CTUe; USEtox [LCIA]
------------------------------------------------------------


TypeError: a float is required

In [62]:
for v in F.fg_lcia(frag.termination(None)._process_ref).values():
    print('%s' % v)

In [7]:
F.terminate_fragment(8, frag)

CatalogError: 0 sources found for 8!

In [60]:
frag.serialize()

{'Comment': '',
 'Direction': 'Input',
 'Flow': 'Polyethylene terephthalate, resin, at plant',
 'Name': 'USLCI - PET',
 'StageName': '',
 'direction': 'Input',
 'entityId': '572813aa-b8a3-4034-bff1-ff7e469d8726',
 'entityType': 'fragment',
 'exchangeValues': {0: None, 1: 1.0},
 'externalId': '572813aa-b8a3-4034-bff1-ff7e469d8726',
 'flow': '2a6b644c-519c-32e9-a7d8-521c3011df09',
 'isBackground': False,
 'isBalanceFlow': False,
 'isPrivate': False,
 'origin': 'fragments',
 'parent': 'None',
 'tags': {'Comment': '',
  'Direction': 'Input',
  'Flow': 'Polyethylene terephthalate, resin, at plant',
  'Name': 'USLCI - PET',
  'StageName': ''},
 'terminations': {None: {'entityId': '42550686-82c9-3e19-b47d-fadc0037f6a8',
   'source': '/data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz'}}}

In [9]:
F.load('USLCI')

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz:
Found Extension: zip
19 new quantity entities added (19 total)
4176 new flow entities added (4176 total)
701 new process entities added (701 total)


In [25]:
F[0].flows()[0]['Compartment']

['Chemical Manufacturing', 'All Other Basic Organic Chemical Manufacturing']

In [22]:
frag = [f for f in F[0].fragments()][0]

In [23]:
frag.flow['Compartment']

['Chemical Manufacturing', 'Petrochemical Manufacturing']

In [26]:
F.save()

Saving foreground


In [3]:
pick_one(F[0].flows()).entity_type

(selecting only choice)


'flow'

In [3]:
F.create_fragment()

Create fragment.
Name: USLCI - PET
Reference flow: Use (F)oreground flow or (S)earch for flow? [FS] [F]: 
Select Reference flow:
(selecting only choice)
Direction w.r.t. upstream:

Choice Item
====== ======================================================================
  (I)  Input
  (O)  Output
------ ----------------------------------------------------------------------
Enter choice (or "None"): I
interface
name: USLCI - PET
flow: Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing]
dirn: Input
LcFragment
Name: USLCI - PET
args:
<class 'lcatools.entities.LcFlow'> | Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing]
<class 'str'> | Input
Polyethylene terephthalate, resin, at plant [Chemical Manufacturing, Petrochemical Manufacturing]
<class 'lcatools.entities.LcFlow'>


In [19]:
frag = next(fr for fr in F[0]._entities_by_type('fragment'))

In [20]:
frag.serialize()

{'Comment': '',
 'Direction': 'Input',
 'Flow': 'Polyethylene terephthalate, resin, at plant',
 'Name': 'USLCI - PET',
 'StageName': '',
 'direction': 'Input',
 'entityId': '572813aa-b8a3-4034-bff1-ff7e469d8726',
 'entityType': 'fragment',
 'exchangeValues': {0: None, 1: 1.0},
 'externalId': '572813aa-b8a3-4034-bff1-ff7e469d8726',
 'flow': '2a6b644c-519c-32e9-a7d8-521c3011df09',
 'isBackground': False,
 'isBalanceFlow': False,
 'isPrivate': False,
 'origin': 'fragments',
 'parent': 'None',
 'tags': {'Comment': '',
  'Direction': 'Input',
  'Flow': 'Polyethylene terephthalate, resin, at plant',
  'Name': 'USLCI - PET',
  'StageName': ''},
 'terminations': {}}

In [21]:
F.save()

Saving foreground


In [18]:
F.load('USLCI')

Loading JSON data from /data/GitHub/lca-tools-datafiles/catalogs/uslci_ecospold.json.gz:
Found Extension: zip
19 new quantity entities added (19 total)
4176 new flow entities added (4176 total)
701 new process entities added (701 total)


In [ ]:
F.terminate('USLCI', )

In [12]:
from lcatools.foreground.fragment_flows import LcFragment

In [13]:
selected = selected[3]

In [18]:
show([selected])

[ 0] (USLCI) {f} CUTOFF Polyethylene terephthalate, granulate, amorphous, at plant [CUTOFF Flows] 


In [20]:
frag = LcFragment.new('PETfrom USLCI', selected.entity(), 'Input')

In [22]:
frag.__dict__

{'_background': False,
 '_balance_flow': False,
 '_cached_unit_scores': {},
 '_conserved_quantity': None,
 '_d': {'Comment': '',
  'Direction': 'Input',
  'Flow': 'CUTOFF Polyethylene terephthalate, granulate, amorphous, at plant',
  'Name': 'PETfrom USLCI',
  'StageName': ''},
 '_exchange_values': {0: None, 1: 1.0},
 '_external_ref': None,
 '_origin': None,
 '_private': False,
 '_scenarios': {},
 '_terminations': {},
 '_uuid': UUID('e0d04a0e-7307-424b-8bcb-fbaddea01382'),
 'direction': 'Input',
 'entity_type': 'fragment',
 'flow': <lcatools.entities.LcFlow at 0x7f4fe1d07710>,
 'observed_magnitude': 1.0,
 'reference_entity': None}

In [24]:
'%s' % frag.flow

'CUTOFF Polyethylene terephthalate, granulate, amorphous, at plant [CUTOFF Flows]'

In [27]:
frag._parent

In [25]:
frag.serialize()

AttributeError: 'NoneType' object has no attribute 'get_uuid'